In [112]:
import pandas as pd
import glob
pd.options.mode.chained_assignment = None

In [113]:
# search all files inside a specific folder
# *.* means file name with any extension
dir_path = r'*.parquet'
data_files = glob.glob(dir_path)
datafiles = []
for f in data_files:
    if (f.find("_dbsn") == -1):
        datafiles.append(pd.read_parquet(f))
data_igm_osm = pd.concat(datafiles)
data_igm_osm = data_igm_osm.fillna(0).reset_index()
del data_igm_osm['index']
columns = data_igm_osm.columns.unique()

In [114]:
bad_columns = []
for column in columns:
    if column.find("=>") > -1:
        bad_columns.append(column)

In [115]:
for bad in bad_columns:
    c = data_igm_osm[data_igm_osm[bad] > 0].shape[0]
    if c > 0:
        investigate = data_igm_osm[data_igm_osm[bad] >0]
        pp = list(investigate.province.unique())
        for p in pp:
            total = investigate[investigate.province == p][bad].values[0]
            if bad.find("buildings_source") > -1:
                nv = data_igm_osm[data_igm_osm.province == p]["buildings_source_03"].values[0] + total
                data_igm_osm[data_igm_osm.province == p]["buildings_source_03"] = nv
            if bad.find("streets_source") > -1:
                nv = data_igm_osm[data_igm_osm.province == p].streets_source_03.values[0] + total
                data_igm_osm[data_igm_osm.province == p].streets_source_03 = nv                


In [116]:
# conversion in km
# osm_in_igm_buildings_area
data_igm_osm.osm_in_igm_buildings_area = data_igm_osm.osm_in_igm_buildings_area / 1000
# length_osm_in_igm
data_igm_osm.length_osm_in_igm = data_igm_osm.length_osm_in_igm / 1000
# buildings_igm_area
data_igm_osm.buildings_igm_area = data_igm_osm.buildings_igm_area / 1000
# buildings_osm_area
data_igm_osm.buildings_osm_area = data_igm_osm.buildings_osm_area / 1000
# streets_osm_meters
data_igm_osm.streets_osm_meters = data_igm_osm.streets_osm_meters / 1000
# streets_igm_meters
data_igm_osm.streets_igm_meters = data_igm_osm.streets_igm_meters / 1000
# area_buildings_from_osm
data_igm_osm.area_buildings_from_osm = data_igm_osm.area_buildings_from_osm / 1000
# length_streets_from_osm
data_igm_osm.length_streets_from_osm = data_igm_osm.length_streets_from_osm / 1000


In [117]:
data_igm_osm.rename(columns={"streets_osm_meters":"streets_osm_km","streets_igm_meters":"streets_igm_km"},inplace=True)

In [118]:
exclude_columns = ['streets_source_01', 'streets_source_04', 'buildings_source_02',
       'buildings_source_04', 'buildings_source_0504', 'province', 'region',
       'streets_source_02', 'streets_source_03', 'buildings_source_01',
       'buildings_source_03', 'streets_source_0101', 'streets_source_95',
       'streets_source_0503', 'buildings_source_0101', 'buildings_source_10',
       'buildings_source_05', 'streets_source_23', 'buildings_source_07',
       'buildings_source_09', 'streets_source_07', 'buildings_source_20',
       'streets_source_05', 'buildings_source_23', 'buildings_source_21']

In [119]:
include_columns = ['osm_in_igm_buildings', 'osm_buildings', 'osm_igm',
       'osm_in_igm_buildings_area', 'osm_in_igm_streets', 'length_osm_in_igm',
       'osm_streets', 'igm_streets', 'buildings_igm_area',
       'buildings_osm_area', 'streets_osm_km', 'streets_igm_km',
       'area_buildings_from_osm', 'length_streets_from_osm']

In [120]:
data_igm_osm_nosource = data_igm_osm[include_columns]

In [121]:
df1 = data_igm_osm.groupby("region")['buildings_igm_area'].sum().to_frame().reset_index()
df2 = data_igm_osm.groupby("region")['buildings_osm_area'].sum().to_frame().reset_index()
df3 = data_igm_osm.groupby("region")['area_buildings_from_osm'].sum().to_frame().reset_index()
df4 = data_igm_osm.groupby("region")['osm_in_igm_buildings_area'].sum().to_frame().reset_index()

In [122]:
df5 = data_igm_osm.groupby("region")['streets_igm_km'].sum().to_frame().reset_index()
df6 = data_igm_osm.groupby("region")['streets_osm_km'].sum().to_frame().reset_index()
df7 = data_igm_osm.groupby("region")['length_osm_in_igm'].sum().to_frame().reset_index()
df8 = data_igm_osm.groupby("region")['length_streets_from_osm'].sum().to_frame().reset_index()

In [123]:
regions = df1.merge(df2).merge(df3).merge(df4).merge(df5).merge(df6).merge(df7).merge(df8)

In [124]:
# qui sicuramente qualcosa non funziona
regions.area_buildings_from_osm[1] = regions.area_buildings_from_osm[1][0]
regions.area_buildings_from_osm[2] = regions.area_buildings_from_osm[2][0]
regions.area_buildings_from_osm[4] = regions.area_buildings_from_osm[4][0]
regions.area_buildings_from_osm[7] = regions.area_buildings_from_osm[7][0]
regions.area_buildings_from_osm[8] = regions.area_buildings_from_osm[8][0]
regions.area_buildings_from_osm[9] = regions.area_buildings_from_osm[9][0]
regions.area_buildings_from_osm[11] = regions.area_buildings_from_osm[11][0]

In [125]:
regions

,region,buildings_igm_area,buildings_osm_area,area_buildings_from_osm,osm_in_igm_buildings_area,streets_igm_km,streets_osm_km,length_osm_in_igm,length_streets_from_osm
0,Abruzzo,117867.192254,44398.720824,330.485123,47795.426988,56224.277260,43849.626198,39557.270560,1.154174
1,Basilicata,52368.064443,20667.007065,8.415432,52762.469523,46544.688942,31358.487399,29404.122716,167.793594
2,Calabria,103275.990957,33555.635669,18.017347,59853.356660,58671.935687,27046.850675,25291.786015,3.833126
3,Campania,312199.886372,144879.448646,83.938698,251645.030162,92203.243370,60151.279027,54703.519215,31873.686935
4,Lazio,229187.084550,174616.283164,240.491812,186168.942781,62743.592272,53113.286825,44976.856439,38.607436
5,Marche,111698.280263,48542.403588,1486.506571,74116.241049,43841.060843,39948.055108,34027.792243,1664.780028
6,Molise,26778.195031,12267.506022,59.694779,11947.016744,17007.671043,14855.614153,13812.103779,1593.493095
7,Puglia,272849.931395,290203.906476,153.154035,311295.666965,109238.717754,84073.764284,75856.629197,607.558142
8,Sardegna,119449.428179,95144.368901,1.047049,98547.495375,112557.516506,67537.336734,62728.643029,707.447273
9,Sicilia,331263.201403,141684.342000,711.550907,215186.496347,142505.842003,100172.973171,90578.903064,13906.169106


In [126]:
regions.to_csv("regions_igm.csv",index=False)